In [1]:
#Import needed packages
import torch
import torch.nn as nn
from torchvision import datasets, models
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import numpy as np
from torch import optim

import json


In [2]:
class Unit(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        

        self.conv = nn.Conv2d(in_channels=in_channels,kernel_size=3,out_channels=out_channels,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self,input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output

class SimpleNet(nn.Module):
    def __init__(self,num_classes=10):
        super(SimpleNet,self).__init__()

        #Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        #Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 ,self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)

        self.fc = nn.Linear(in_features=128,out_features=num_classes)

    def forward(self, input):
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        return output


In [3]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

means = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {'training' : transforms.Compose([transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.RandomRotation(90),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(means, std)]),
                   
                   'validation' : transforms.Compose([transforms.Resize(224),
                                                      transforms.CenterCrop(224),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(means, std)]),
                   
                   'testing' : transforms.Compose([transforms.Resize(224),
                                                   transforms.CenterCrop(224),
                                                   transforms.ToTensor(),
                                                   transforms.Normalize(means, std)])                                     
                }

image_datasets = {'training' : datasets.ImageFolder(train_dir,transform=data_transforms['training']),
                  'validation' : datasets.ImageFolder(valid_dir,transform=data_transforms['validation']),
                  'testing' : datasets.ImageFolder(test_dir,transform=data_transforms['testing'])
                 }

dataloaders = {'training' : torch.utils.data.DataLoader(image_datasets['training'],batch_size= 128, shuffle= True),
               'validation' : torch.utils.data.DataLoader(image_datasets['validation'],batch_size= 32, shuffle= True),
               'testing' : torch.utils.data.DataLoader(image_datasets['testing'],batch_size= 32, shuffle= True)
              }

In [4]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
class_to_index = image_datasets['training'].class_to_idx
class_from_index = dict([val,key] for key,val in class_to_index.items())

print(class_from_index)

{0: '1', 1: '10', 2: '100', 3: '101', 4: '102', 5: '11', 6: '12', 7: '13', 8: '14', 9: '15', 10: '16', 11: '17', 12: '18', 13: '19', 14: '2', 15: '20', 16: '21', 17: '22', 18: '23', 19: '24', 20: '25', 21: '26', 22: '27', 23: '28', 24: '29', 25: '3', 26: '30', 27: '31', 28: '32', 29: '33', 30: '34', 31: '35', 32: '36', 33: '37', 34: '38', 35: '39', 36: '4', 37: '40', 38: '41', 39: '42', 40: '43', 41: '44', 42: '45', 43: '46', 44: '47', 45: '48', 46: '49', 47: '5', 48: '50', 49: '51', 50: '52', 51: '53', 52: '54', 53: '55', 54: '56', 55: '57', 56: '58', 57: '59', 58: '6', 59: '60', 60: '61', 61: '62', 62: '63', 63: '64', 64: '65', 65: '66', 66: '67', 67: '68', 68: '69', 69: '7', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75', 76: '76', 77: '77', 78: '78', 79: '79', 80: '8', 81: '80', 82: '81', 83: '82', 84: '83', 85: '84', 86: '85', 87: '86', 88: '87', 89: '88', 90: '89', 91: '9', 92: '90', 93: '91', 94: '92', 95: '93', 96: '94', 97: '95', 98: '96', 99: '97', 100: '98', 101:

In [5]:
def get_class_name(class_from_index):
    return cat_to_name[class_from_index]

In [6]:
model = SimpleNet(num_classes=102)

In [7]:
def test_accuracy(model,dataloader,criterion):
    current_loss = 0
    accuracy = 0
    model.eval()
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)       
        
        output = model.forward(images)
        current_loss += criterion(output,labels).item()
        
        ps = torch.exp(output)
                   
        # Each probability consist in an array of probabilities, the predicted class is the
        # highest probability of each one, so we need to get the max of the probabilitie for each 
        # prediction in order to find the predicted_class         
        
        predicted_classes = ps.max(dim=1)[1]
        
        # To find the correct classified items we compare labels with the predicted_classes
        # and we will get back an array of booleans with 1 if the prediction was correct or
        # 0 otherwise
                
        correct_predictions = (labels.data == predicted_classes)

        accuracy += correct_predictions.type(torch.FloatTensor).mean()
        
    return current_loss, accuracy

In [8]:
learning_rate = 0.0015

criterion = nn.NLLLoss()
adam_optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
def train_network(model):
# Train Network
    epochs = 15
    print_every = 20
    step = 0
    running_loss = 0
    validation_data = dataloaders['validation']
    len_val = len(validation_data)

    model = model.to('cpu')    

    
    for epoch in range(epochs):
        running_loss = 0    
        model.train()

        for inputs, y in dataloaders['training']:
            step += 1
            adam_optimizer.zero_grad()
            inputs, y = inputs.to('cpu'), y.to('cpu')

            y_hat = model.forward(inputs)

            loss = criterion(y_hat,y)
            loss.backward()

            adam_optimizer.step()

            running_loss += loss.item()

            if step %  print_every == 0:
                model.eval()

                with torch.no_grad():
                    test_loss, accuracy = test_accuracy(model,validation_data,criterion)

                print("Epoch: {}/{}... ".format(epoch+1, epochs),
                      "Loss: {:.3f}".format(running_loss/print_every),
                      "Test Loss: {:.3f}".format(test_loss/len_val),
                      "Test Accuracy: {:.3f}".format(accuracy/len_val))

                running_loss = 0
                model.train()

    train_executed = True

In [10]:
# TODO: Do validation on the test set
train_executed = False
model_validated = False
def validate_trained_model(dataloader):
    _, accuracy = test_accuracy(densenet,dataloader,criterion)
    print('Accuracy in the validation set is: {} %'.format(round(float(accuracy/len(dataloader)),2)*100))
    #print('Accuracy in the validation set is: {} '.format(accuracy/len(dataloaders['validation'])))
    
# DONE: Do validation on the test set
if train_executed:    
    validate_trained_model(dataloaders['testing'])
    model_validated =True
else:
    print('Model was not trained yet, check the model accuracy below after loading the checkpoint')

Model was not trained yet, check the model accuracy below after loading the checkpoint


In [11]:
train_network(model=model)

RuntimeError: $ Torch: not enough memory: you tried to allocate 6GB. Buy new RAM! at /pytorch/aten/src/TH/THGeneral.cpp:204